<a href="https://colab.research.google.com/github/jncln/CCMACLRL_EXERCISES_COM222ML/blob/main/Exercise7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 7: Hate Speech Classification using Multinomial Naive Bayes

Instructions:
- You do not need to split your data. Use the training, validation and test sets provided below.
- Use Multinomial Naive Bayes to train a model that can classify if a sentence is a hate speech or non-hate speech
- A sentence with a label of zero (0) is classified as non-hate speech
- A sentence with a label of one (1) is classified as a hate speech

Apply text pre-processing techniques such as
- Converting to lowercase
- Stop word Removal
- Removal of digits, special characters
- Stemming or Lemmatization but not both
- Count Vectorizer or TF-IDF Vectorizer but not both

Evaluate your model by:
- Providing input by yourself
- Creating a Confusion Matrix
- Calculating the Accuracy, Precision, Recall and F1-Score

In [139]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nltk
import seaborn as sns
import re
import os, types

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, precision_score, recall_score, accuracy_score, balanced_accuracy_score, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download("punkt")
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [140]:
splits = {'train': 'unique_train_dataset.csv', 'validation': 'unique_validation_dataset.csv', 'test': 'unique_test_dataset.csv'}

In [141]:

filipino_stopwords = set(['tangina', 'pangit', 'leche ka boybie', 'tanginamo lawrence'])


**Training Set**

Use this to train your model

In [142]:
df_train = pd.read_csv("hf://datasets/mapsoriano/2016_2022_hate_speech_filipino/" + splits["train"])

**Validation Set**

Use this set to evaluate your model

In [143]:
df_validation = pd.read_csv("hf://datasets/mapsoriano/2016_2022_hate_speech_filipino/" + splits["validation"])

**Test Set**
  
Use this set to test your model

In [144]:
df_test = pd.read_csv("hf://datasets/mapsoriano/2016_2022_hate_speech_filipino/" + splits["test"])

In [145]:
df_train.head()

,text,label
0,Presidential candidate Mar Roxas implies that ...,1
1,Parang may mali na sumunod ang patalastas ng N...,1
2,Bet ko. Pula Ang Kulay Ng Posas,1
3,[USERNAME] kakampink,0
4,Bakit parang tahimik ang mga PINK about Doc Wi...,1


## A. Understanding your training data

1. Check the first 10 rows of the training dataset

In [149]:
print(df_train.head(10))

                                                text  label
0  Presidential candidate Mar Roxas implies that ...      1
1  Parang may mali na sumunod ang patalastas ng N...      1
2                    Bet ko. Pula Ang Kulay Ng Posas      1
3                               [USERNAME] kakampink      0
4  Bakit parang tahimik ang mga PINK about Doc Wi...      1
5  "Ang sinungaling sa umpisa ay sinungaling hang...      1
6                                          Leni Kiko      0
7  Nahiya si Binay sa Makati kaya dito na lang sa...      1
8                            Another reminderHalalan      0
9  [USERNAME] Maybe because VP Leni Sen Kiko and ...      0


2. Check how many rows and columns are in the training dataset using `.info()`

In [174]:
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21773 entries, 0 to 21772
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    21773 non-null  object
 1   label   21773 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 340.3+ KB
None


3. Check for NaN values

In [175]:
print(df_train.isna().sum())

text     0
label    0
dtype: int64


4. Check for duplicate rows

In [176]:
print(df_train.duplicated().sum())

125


5. Check how many rows belong to each class

In [177]:
print(df_train['label'].value_counts())

label
1    10994
0    10779
Name: count, dtype: int64


## B. Text pre-processing

6. Remove duplicate rows

In [178]:
df_train.drop_duplicates

<bound method DataFrame.drop_duplicates of                                                     text  label
0      presidential candidate mar roxas implies govt ...      1
1      parang may mali na sumunod ang patalastas ng n...      1
2                       bet ko . pula ang kulay ng posas      1
3                                 [ username ] kakampink      0
4      bakit parang tahimik ang mga pink doc willie o...      1
...                                                  ...    ...
21768                   marcos talunan marcos magnanakaw      1
21769           grabe kayo kay binay ? ? ? ? ? ? ? ? ? ?      0
21770  [ username ] cnu ba naman ang hindimabibighani...      0
21771  rt [ username ] : tabi tabi yung mga nagsasabi...      1
21772  honestly dont get kakampinks like pano nila na...      1

[21773 rows x 2 columns]>

7. Remove rows with NaN values

In [179]:
df_train.dropna(inplace=True)

8. Convert all text to lowercase

In [180]:
df_train['text'] = df_train['text'].str.lower()

9. Remove digits, URLS and special characters

In [181]:
df_train['text'] = df_train['text'].str.replace(r'http\S+', '', regex=True)

10. Remove stop words

In [158]:
df_train['text'] = df_train['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords.words('english'))]))

11. Use Stemming or Lemmatization

In [159]:
lemmatizer = WordNetLemmatizer()
df_train['text'] = df_train['text'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in word_tokenize(x)]))

## C. Training your model

12. Put all text training data in variable **X_train**

In [160]:
X_train = df_train['text']

13. Put all training data labels in variable **y_train**

In [161]:
y_train = df_train['label']

14. Use `CountVectorizer()` or `TfidfVectorizer()` to convert text data to its numerical form.

Put the converted data to **X_train_transformed** variable

In [162]:
vectorizer = TfidfVectorizer()
X_train_transformed = vectorizer.fit_transform(X_train)

15. Create an instance of `MultinomalNB()`

In [163]:
model = MultinomialNB()

16. Train the model using `.fit()`

In [164]:
model.fit(X_train_transformed, y_train)

MultinomialNB()

## D. Evaluate your model

17. Use `.predict()` to generate model predictions using the **validation dataset**


- Put all text validation data in **X_validation** variable

- Convert **X_validation** to its numerical form.

- Put the converted data to **X_validation_transformed**

- Put all predictions in **y_validation_pred** variable

In [165]:
X_test = df_test['text']
X_test = X_test.str.lower()
X_test = X_test.apply(lambda x: re.sub(r'\d+', '', x))
X_test = X_test.apply(lambda x: re.sub(r'http\S+|www.\S+', '', x))
X_test = X_test.apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
X_test = X_test.apply(lambda x: ' '.join([word for word in word_tokenize(x) if filipino_stopwords]))
X_test = X_test.apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in word_tokenize(x)]))

X_test_transformed = vectorizer.transform(X_test)

18. Get the Accuracy, Precision, Recall and F1-Score of the model using the **validation dataset**

- Put all validation data labels in **y_validation** variable

In [166]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

vectorizer = TfidfVectorizer()
X_train_transformed = vectorizer.fit_transform(df_train['text'])

X_test_transformed = vectorizer.transform(df_test['text'])

model = MultinomialNB()
model.fit(X_train_transformed, df_train['label'])

y_test = df_test['label']
y_test_pred = model.predict(X_test_transformed)

accuracy_test = accuracy_score(y_test, y_test_pred)
precision_test = precision_score(y_test, y_test_pred, average='weighted')
recall_test = recall_score(y_test, y_test_pred, average='weighted')
f1_test = f1_score(y_test, y_test_pred, average='weighted')

print(f"Test Accuracy: {accuracy_test}")
print(f"Test Precision: {precision_test}")
print(f"Test Recall: {recall_test}")
print(f"Test F1 Score: {f1_test}")


print(f"Shape of training data: {X_train_transformed.shape}")
print(f"Shape of test data: {X_test_transformed.shape}")


Test Accuracy: 0.8295373665480427
Test Precision: 0.831043059720294
Test Recall: 0.8295373665480427
Test F1 Score: 0.8293786325367034
Shape of training data: (21773, 36448)
Shape of test data: (2810, 36448)


19. Create a confusion matrix using the **validation dataset**

In [167]:
cm = confusion_matrix(y_validation, y_validation_pred)
print(f"Confusion Matrix:\n{cm}")

Confusion Matrix:
[[1723  416]
 [ 302 1914]]


20. Use `.predict()` to generate the model predictions using the **test dataset**


- Put all text validation data in **X_test** variable

- Convert **X_test** to its numerical form.

- Put the converted data to **X_test_transformed**

- Put all predictions in **y_test_pred** variable

In [168]:
X_test = df_test['text']
X_test = X_test.str.lower()
X_test = X_test.apply(lambda x: re.sub(r'\d+', '', x))
X_test = X_test.apply(lambda x: re.sub(r'http\S+|www.\S+', '', x))
X_test = X_test.apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
X_test = X_test.apply(lambda x: ' '.join([word for word in word_tokenize(x) if word not in filipino_stopwords]))
X_test = X_test.apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in word_tokenize(x)]))

X_test_transformed = vectorizer.transform(X_test)

21. Get the Accuracy, Precision, Recall and F1-Score of the model using the **test dataset**

- Put all test data labels in **y_validation** variable



In [169]:
y_test = df_test['label']
y_test_pred = model.predict(X_test_transformed)

accuracy_test = accuracy_score(y_test, y_test_pred)
precision_test = precision_score(y_test, y_test_pred)
recall_test = recall_score(y_test, y_test_pred)
f1_test = f1_score(y_test, y_test_pred)

print(f"Test Accuracy: {accuracy_test}")
print(f"Test Precision: {precision_test}")
print(f"Test Recall: {recall_test}")
print(f"Test F1 Score: {f1_test}")

Test Accuracy: 0.8238434163701067
Test Precision: 0.8069340584636302
Test Recall: 0.8490701001430615
Test F1 Score: 0.8274660160334611


22. Create a confusion matrix using the **test dataset**

In [170]:
cm_test = confusion_matrix(y_test, y_test_pred)
print(f"Test Confusion Matrix:\n{cm_test}")

Test Confusion Matrix:
[[1128  284]
 [ 211 1187]]


## E. Test the model

23. Test the model by providing a non-hate speech input. The model should predict it as 0

In [191]:
input_non_hate = ["magandang gabi po sir ponio "]
input_non_hate_transformed = vectorizer.transform(input_non_hate)
prediction_non_hate = model.predict(input_non_hate_transformed)

In [192]:
print(f"Prediction for non-hate input: {prediction_non_hate[0]}")

Prediction for non-hate input: 0


24. Test the model by providing a hate speech input. The model should predict it as 1

In [186]:
input_hate = ["putanginaaaa ayokooo na pagod na pagod na ako"]
input_hate_transformed = vectorizer.transform(input_hate)
prediction_hate = model.predict(input_hate_transformed)

print(f"Prediction for hate input: {prediction_hate[0]}")

Prediction for hate input: 1
